In [ ]:
# SpeechBrain là một thư viện mạnh mẽ và dễ dùng cho các tác vụ Xử lý tiếng nói bằng AI
# Bộ công cụ phần mềm đã xây dựng sẵn, giúp bạn làm việc hiệu quả hơn trong một lĩnh vực cụ thể như:
# AI, xử lý tiếng nói, thị giác máy tính, xử lý ngôn ngữ...

In [ ]:
# Cài đúng phiên bản tương thích với GPU Kaggle
!pip install --no-cache-dir torch==2.5.1 torchaudio==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu124
# Cài lại SpeechBrain
!pip install --no-cache-dir speechbrain

In [ ]:
import speechbrain
print(speechbrain.__version__)


In [ ]:
#SpeechBrain là toolkit đã tích hợp sẵn ECAPA‑TDNN và utilities fine‑tune 
# os: làm việc với hệ điều hành
# torchaudio: xử lý âm thành
# pandas: xử lý bảng dữ liệu

import os, torchaudio, pandas as pd
from pathlib import Path
base = Path("/kaggle/input/vietnam-celeb-dataset/full-dataset")


In [ ]:
# (2)
# Mục Đích  Chuyển đổi file (cleaned_utterances (1).txt) thành một file train_manifest.csv có 2 cột: wav, spk_id
# Là chuyển đổi dữ liệu thô dạng .txt (chứa danh sách đường dẫn .wav) thành một file 
#.csv có cấu trúc rõ ràng (gồm cột wav và spk_id), để phục vụ cho việc huấn luyện mô hình.
# một bước tiền xử lý rất quan trọng trước khi fine-tune mô hình ECAPA-TDNN: 
# tách tập huấn luyện và tập validation từ file train_manifest.csv.

from pathlib import Path
import pandas as pd

train_txt = "/kaggle/input/clean-utterances/cleaned_utterances (1).txt"
base      = Path("/kaggle/input/vietnam-celeb-dataset/full-dataset")

train_rows = []

with open(train_txt) as f:
    for line in f:
        parts = line.strip().split()           # tách bằng mọi khoảng trắng / tab
        if len(parts) == 1:
            # đã dạng id00000/00000.wav
            rel = parts[0]
        elif len(parts) == 2:
            # dạng "id00000 00000.wav" hoặc tab
            rel = f"{parts[0]}/{parts[1]}"
        else:
            # dòng lạ → bỏ qua
            continue

        wav_path = base / "data" / rel
        if wav_path.exists():
            spk = rel.split('/')[0]            # id00000
            train_rows.append([str(wav_path), spk])

print("Số file WAV hợp lệ:", len(train_rows))

pd.DataFrame(train_rows, columns=["wav", "spk_id"]).to_csv(
    "train_manifest.csv", index=False)

In [ ]:
# (3)
# Trộn ngẫu nhiên và chia tập train/validation
# không được đánh giá mô hình trên chính tập dữ liệu mà bạn đã dùng để huấn luyện.
# Nếu làm vậy, mô hình sẽ chỉ "học thuộc" (memorize) thay vì học khái quát (generalize).
import pandas as pd

# 1) Đọc train_manifest.csv
df = pd.read_csv("train_manifest.csv")
# Một DataFrame giống như bảng dữ liệu dạng Excel, với hàng và cột.

# 2) Trộn ngẫu nhiên và chia
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
n_train = int(len(df) * 0.9)

train_df = df.iloc[:n_train]
valid_df = df.iloc[n_train:]

# 3) Ghi hai file mới vào working dir
train_df.to_csv("train_manifest.csv", index=False) # dùng để train model
valid_df.to_csv("valid_manifest.csv", index=False) # kiểm tra model khi train

print(f"Đã tạo: train_manifest.csv ({len(train_df)} samples)  |  valid_manifest.csv ({len(valid_df)} samples)")
# tạo được 2 file .csv trong thư mục làm việc (working directory)

In [ ]:
#(4)   CLEAN manifest + ép kiểu spk_lbl + thêm ID
import pandas as pd, os

def fix_manifest(csv_in, csv_out=None):
    if csv_out is None:
        csv_out = csv_in
    df = pd.read_csv(csv_in)

    # Giữ file WAV tồn tại
    df = df[df["wav"].apply(os.path.exists)].reset_index(drop=True)

    # Nếu thiếu spk_lbl: tạo từ spk_id
    if "spk_lbl" not in df.columns:
        spk2idx = {spk: i for i, spk in enumerate(sorted(df.spk_id.unique()))}
        df["spk_lbl"] = df.spk_id.map(spk2idx)

    # 👉 Luôn ép về numeric, bỏ hàng rỗng, ép int
    df["spk_lbl"] = pd.to_numeric(df["spk_lbl"], errors="coerce")
    df = df.dropna(subset=["spk_lbl"]).reset_index(drop=True)
    df["spk_lbl"] = df["spk_lbl"].astype(int)

    # Thêm ID nếu thiếu
    if "ID" not in df.columns:
        df.insert(0, "ID", range(len(df)))

    df.to_csv(csv_out, index=False)
    print(f"{csv_out} ✅  {len(df)} rows")

# chạy cho cả train & valid
fix_manifest("train_manifest.csv")
fix_manifest("valid_manifest.csv")


In [ ]:
# (5)
# ==== NEW CELL: làm sạch manifest & gắn nhãn số ====
# lọc bỏ đường dẫn WAV không tồn tại
# map spk_id (string) ↦ spk_lbl (int) để dùng cross‑entropy.

import pandas as pd, os

def load_clean(csv_path):
    """Đọc CSV, bỏ WAV thiếu & strip khoảng trắng spk_id."""
    df = pd.read_csv(csv_path)
    df = df[df["wav"].apply(os.path.exists)].reset_index(drop=True)
    df["spk_id"] = df["spk_id"].astype(str).str.strip()
    return df

train_df = load_clean("train_manifest.csv")
valid_df = load_clean("valid_manifest.csv")

# 1) map spk_id -> index (chỉ dựa trên TRAIN)
spk2idx = {spk: i for i, spk in enumerate(sorted(train_df.spk_id.unique()))}
train_df["spk_lbl"] = train_df.spk_id.map(spk2idx)

# 2) Giữ VALID chỉ gồm speaker đã có trong TRAIN
valid_df = valid_df[valid_df.spk_id.isin(spk2idx)].reset_index(drop=True)
valid_df["spk_lbl"] = valid_df.spk_id.map(spk2idx)

# 3) Bỏ mọi hàng còn NaN (phòng hờ) & ép kiểu int
train_df = train_df.dropna(subset=["spk_lbl"]).copy()
valid_df = valid_df.dropna(subset=["spk_lbl"]).copy()
train_df["spk_lbl"] = train_df["spk_lbl"].astype(int)
valid_df["spk_lbl"] = valid_df["spk_lbl"].astype(int)

# 4) Lưu lại
train_df.to_csv("train_manifest.csv", index=False)
valid_df.to_csv("valid_manifest.csv", index=False)

num_spk = len(spk2idx)
print(f"Manifest OK ✔  num_spk={num_spk} | "
      f"train={len(train_df)} | valid={len(valid_df)}")

In [ ]:
assert not train_df["spk_lbl"].isna().any()
assert not valid_df["spk_lbl"].isna().any()
print("Không còn spk_lbl trống ✅")


In [ ]:
# (6)
# Bước nạp mô hình ECAPA-TDNN đã được huấn luyện sẵn (pretrained)
# và đóng băng phần "khối não" chính (feature extractor) để thực hiện fine-tuning hiệu quả hơn.
# Nạp ECAPA‑TDNN & đóng băng khối não
# ==== Nạp ECAPA‑TDNN gốc & tạo classifier mới ====
# Nạp ECAPA‑TDNN đã huấn luyện sẵn & gắn classifier cho 1 000 speaker Việt.
# ‑ Chỉ đóng băng 2 block đầu (layer1, layer2) → phần còn lại vẫn fine‑tune.
# ‑ KHÔNG tạo optimizer, KHÔNG tạo checkpointer (sẽ làm ở Cell 8).

from speechbrain.pretrained import SpeakerRecognition
import torch.nn as nn
import os

# 1) Load model ECAPA‑TDNN (SpeechBrain)
pretrained = SpeakerRecognition.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_ecapa"            # cache tại /kaggle/working
)

compute_features = pretrained.mods.compute_features
mean_var_norm   = pretrained.mods.mean_var_norm
embedding_model = pretrained.mods.embedding_model

# 2) Freeze CHỈ hai block đầu → giữ layer3/4, ASP được học tiếp
for name, module in embedding_model.named_children():
    if name in ["layer1", "layer2"]:
        for p in module.parameters():
            p.requires_grad = False

# 3) Classifier mới cho num_spk lớp (num_spk đã tính ở Cell 5)
classifier = nn.Linear(192, num_spk, bias=True)
nn.init.xavier_uniform_(classifier.weight)
nn.init.zeros_(classifier.bias)

print("✅ ECAPA loaded  •  frozen layers: layer1 & layer2  •  new classifier ready")

In [ ]:
# --- wav_collate mới ---
def wav_collate(batch):
    sigs  = [item["sig"] for item in batch]              # list [1,T_i]
    lbls  = torch.tensor([item["lbl"] for item in batch])
    lens  = torch.tensor([s.shape[-1] for s in sigs], dtype=torch.float32)

    max_len = lens.max().int().item()
    padded  = torch.stack([F.pad(s, (0, max_len - s.shape[-1])) for s in sigs])

    # trả về TỰ NHIÊN: lens (số frame), không chia max_len
    return {"sig": (padded, lens), "lbl": lbls}



In [ ]:
# (7) ───────────────────────────────────────────────────────────
# ==== DATASET & DATALOADER (đã fix) ====
from speechbrain.dataio.dataset import DynamicItemDataset
from speechbrain.dataio.dataloader import SaveableDataLoader
import torchaudio, torch

MAX_SEC = 8              # giới hạn 8 giây
MAX_SAMPLES = 16000 * MAX_SEC

def audio_pipeline(wav_path):
    sig, _ = torchaudio.load(wav_path)

    # gộp về mono
    if sig.dim() == 2:
        sig = sig.mean(0, keepdim=True)

    # ---- CẮT NGẪU NHIÊN nếu >8s ----
    if sig.shape[-1] > MAX_SAMPLES:
        start = torch.randint(0, sig.shape[-1] - MAX_SAMPLES, (1,)).item()
        sig = sig[:, start:start + MAX_SAMPLES]

    return sig          # [1, T]

def label_pipeline(lbl):
    return torch.tensor(int(lbl))

def make_ds(csv_path):
    ds = DynamicItemDataset.from_csv(csv_path=csv_path)
    ds.add_dynamic_item(audio_pipeline, takes=["wav"],  provides=["sig"])
    ds.add_dynamic_item(label_pipeline, takes=["spk_lbl"], provides=["lbl"])
    ds.set_output_keys(["sig", "lbl"])
    return ds

train_set = make_ds("train_manifest.csv")
valid_set = make_ds("valid_manifest.csv")

# --------------------- giảm batch_size còn 4 ---------------------
train_dl = SaveableDataLoader(train_set, batch_size=4, shuffle=True,
                              collate_fn=wav_collate)
valid_dl = SaveableDataLoader(valid_set, batch_size=4,
                              collate_fn=wav_collate)

print("Dataloader OK  –  train batches:", len(train_dl),
      "| valid:", len(valid_dl))


In [ ]:
# (8) ─────────────────────────────────────────────────────────────────────
# Định nghĩa Fine-tune Brain + huấn luyện 20 epoch
# (đã sửa lỗi “stagefrom …” và thêm logic unfreeze layer3/4 sau epoch 2)

# ---------- Imports và thiết lập thư mục checkpoints ----------
from speechbrain import Brain, Stage
from speechbrain.utils.metric_stats import MetricStats
from speechbrain.utils.checkpoints import Checkpointer
import torch, torch.nn.functional as F
import os

ckpt_dir = "checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)

class FineTuneBrain(Brain):
    def on_fit_start(self):
        super().on_fit_start()
        # Tạo 2 metric stats riêng cho train và valid
        self.train_acc_metric = MetricStats(metric=lambda p, t: (p.argmax(-1) == t).float())
        self.valid_acc_metric = MetricStats(metric=lambda p, t: (p.argmax(-1) == t).float())
        self.unfrozen = False

    def on_stage_start(self, stage, epoch):
        # Logic unfreeze sau epoch 2
        if stage == Stage.TRAIN and epoch == 2 and not self.unfrozen:
            for name, module in embedding_model.named_children():
                if name not in ["layer1", "layer2"]:
                    for p in module.parameters():
                        p.requires_grad = True
            existing = {id(p) for g in self.optimizer.param_groups for p in g["params"]}
            new_params = [p for p in embedding_model.parameters() if p.requires_grad and id(p) not in existing]
            if new_params:
                self.optimizer.add_param_group({"params": new_params, "lr": 1e-4})
            self.unfrozen = True
            print(f">> Unfroze layer3+4+ASP — added {len(new_params)} new params")

    def compute_forward(self, batch, stage=Stage.TRAIN):
        wavs, lens = batch["sig"]
        wavs = wavs.squeeze(1).to(self.device)
        lens = lens.to(self.device)
        feats  = mean_var_norm(compute_features(wavs), lens)
        embeds = embedding_model(feats, lens)
        if embeds.dim() == 3:
            embeds = embeds.mean(dim=1)
        embeds = F.normalize(embeds, p=2, dim=-1)
        logits = classifier(embeds)
        return logits

    def compute_objectives(self, predictions, batch, stage):
        labels = batch["lbl"].to(self.device)
        loss   = F.cross_entropy(predictions, labels)
        batch_size = labels.size(0)
        dummy_ids  = [0] * batch_size

        # Ghi nhận train acc khi stage=TRAIN, valid acc khi stage!=TRAIN
        if stage == Stage.TRAIN:
            self.train_acc_metric.append(dummy_ids, predictions, labels)
        else:
            self.valid_acc_metric.append(dummy_ids, predictions, labels)

        return loss

    def fit_batch(self, batch):
        loss = super().fit_batch(batch)
        # gradient accumulation: bước optimizer sau mỗi 4 mini‐batch
        if (self.step + 1) % 4 == 0:
            self.optimizer.step()
            self.optimizer.zero_grad()
        return loss

    def on_stage_end(self, stage, stage_loss, epoch):
        if stage == Stage.TRAIN:
            train_acc = self.train_acc_metric.summarize("average") * 100
            lr = self.optimizer.param_groups[0]["lr"]
            print(f"Epoch {epoch} • train_acc={train_acc:.2f}% • train_loss={stage_loss:.3f} • lr={lr:.2e}")
            self.train_acc_metric.clear()

        elif stage == Stage.VALID:
            valid_acc = self.valid_acc_metric.summarize("average") * 100
            print(f"Epoch {epoch} • valid_acc={valid_acc:.2f}% • valid_loss={stage_loss:.3f}")
            self.valid_acc_metric.clear()
            self.checkpointer.save_checkpoint()

# ---------- Khởi tạo và gọi fit() ----------
brain = FineTuneBrain(
    modules = {
        "compute_features":   compute_features,
        "mean_var_norm":      mean_var_norm,
        "embedding_model":    embedding_model,
        "classifier":         classifier,
    },
    opt_class    = torch.optim.AdamW,
    hparams      = {"lr": 3e-4, "weight_decay": 1e-4},
    run_opts     = {"device": "cuda", "dtype": "float16"},
    checkpointer = Checkpointer(ckpt_dir),
)

brain.fit(
    range(60),
    train_set=train_dl,
    valid_set=valid_dl
)
